<a href="https://colab.research.google.com/github/ryanvuu/csc-365-lab7/blob/main/csc_365_lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install python-dotenv


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
username = os.getenv("username")

Install libraries

In [3]:
pip install mysql-connector-python


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import getpass
db_password = getpass.getpass()
# print(db_password)

In [6]:
import mysql.connector
conn = mysql.connector.connect(user=username, password=db_password,
                               host='mysql.labthreesixfive.com',
                               database='rvu03')

In [6]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM lab7_rooms")
result = cursor.fetchall()
print(result)

[('AOB', 'Abscond or bolster', 2, 'Queen', 4, Decimal('175.00'), 'traditional'), ('CAS', 'Convoke and sanguine', 2, 'King', 4, Decimal('175.00'), 'traditional'), ('FNA', 'Frugal not apropos', 2, 'King', 4, Decimal('250.00'), 'traditional'), ('HBB', 'Harbinger but bequest', 1, 'Queen', 2, Decimal('100.00'), 'modern'), ('IBD', 'Immutable before decorum', 2, 'Queen', 4, Decimal('150.00'), 'rustic'), ('IBS', 'Interim but salutary', 1, 'King', 2, Decimal('150.00'), 'traditional'), ('MWC', 'Mendicant with cryptic', 2, 'Double', 4, Decimal('125.00'), 'modern'), ('RND', 'Recluse and defiance', 1, 'King', 2, Decimal('150.00'), 'modern'), ('RTE', 'Riddle to exculpate', 2, 'Queen', 4, Decimal('175.00'), 'rustic'), ('TAA', 'Thrift and accolade', 1, 'Double', 2, Decimal('75.00'), 'modern')]


### FR1: Rooms and Rates ###

In [ ]:
pass

# **Main Program**

In [ ]:
def main():
  while True:
    print("Select a Reservation Option:")
    print("[1] Rooms and Rates")
    print("[2] Reservations")
    print("[3] Reservation Cancellation")
    print("[4] Detailed Reservation Information")
    print("[5] Revenue")
    print("[6] Quit")

    choice = input()
    if choice == "1":
      cursor = conn.cursor()
      cursor.execute("""
                      WITH PopTable AS (
                        SELECT lab7_rooms.RoomCode,
                            lab7_rooms.RoomName,
                            lab7_rooms.Beds,
                            lab7_rooms.bedType,
                            lab7_rooms.maxOcc,
                            lab7_rooms.basePrice,
                            lab7_rooms.decor,
                            ROUND(SUM(DATEDIFF(lab7_reservations.Checkout, lab7_reservations.CheckIn)) / 180, 2) AS PopScore
                        FROM lab7_rooms
                        LEFT JOIN lab7_reservations
                            ON lab7_rooms.RoomCode = lab7_reservations.Room
                            AND lab7_reservations.CheckIn >= DATE_SUB(CURDATE(), INTERVAL 180 DAY)
                        GROUP BY lab7_rooms.RoomCode
                      ), RecentReservations AS (
                        SELECT lab7_rooms.RoomCode,
                            MAX(lab7_reservations.CheckIn) AS MostRecentCheckIn,
                            MAX(lab7_reservations.Checkout) AS MostRecentCheckout
                        FROM lab7_rooms
                        INNER JOIN lab7_reservations
                            ON lab7_rooms.RoomCode = lab7_reservations.Room
                        GROUP BY lab7_rooms.RoomCode
                      )
                      SELECT PopTable.RoomCode,
                          PopTable.RoomName,
                          PopTable.Beds,
                          PopTable.bedType,
                          PopTable.maxOcc,
                          PopTable.basePrice,
                          PopTable.decor,
                          PopTable.PopScore,
                          DATE_ADD(RecentReservations.MostRecentCheckout, INTERVAL 1 DAY) AS NextAvailableCheckIn,
                          DATEDIFF(RecentReservations.MostRecentCheckout, RecentReservations.MostRecentCheckIn) AS MostRecentDuration
                      FROM PopTable
                      INNER JOIN RecentReservations
                          ON PopTable.RoomCode = RecentReservations.RoomCode
                      ORDER BY PopTable.PopScore DESC;
                     """)
      result = cursor.fetchall()
      print(result)
    elif choice == "2":
      pass
    elif choice == "3":
      pass
    elif choice == "4":
      pass
    elif choice == "5":
      pass
    elif choice == "6":
      break


In [8]:
if __name__ == "__main__":
    main()

Select a Reservation Option:
[1] Rooms and Rates
[2] Reservations
[3] Reservation Cancellation
[4] Detailed Reservation Information
[5] Revenue
[6] Quit
[('TAA', 'Thrift and accolade', 1, 'Double', 2, Decimal('75.00'), 'modern', Decimal('0.12'), datetime.date(2024, 7, 13), 12), ('IBD', 'Immutable before decorum', 2, 'Queen', 4, Decimal('150.00'), 'rustic', Decimal('0.11'), datetime.date(2024, 7, 2), 1), ('HBB', 'Harbinger but bequest', 1, 'Queen', 2, Decimal('100.00'), 'modern', Decimal('0.10'), datetime.date(2024, 7, 7), 6), ('FNA', 'Frugal not apropos', 2, 'King', 4, Decimal('250.00'), 'traditional', Decimal('0.09'), datetime.date(2024, 7, 5), 4), ('RND', 'Recluse and defiance', 1, 'King', 2, Decimal('150.00'), 'modern', Decimal('0.09'), datetime.date(2024, 7, 3), 2), ('AOB', 'Abscond or bolster', 2, 'Queen', 4, Decimal('175.00'), 'traditional', Decimal('0.08'), datetime.date(2024, 7, 2), 3), ('RTE', 'Riddle to exculpate', 2, 'Queen', 4, Decimal('175.00'), 'rustic', Decimal('0.08'), 